In [2]:
# Importing core dependencies
import pandas as pd
import numpy as np
import citipy.citipy as cp

# Importing API dependencies
import requests
from ../config" import weather_api_key

SyntaxError: invalid syntax (2320703729.py, line 8)

In [2]:
# Randomly generating 2000 latitudes and longitudes
coords = list(
    zip(
        np.random.uniform(-90, 89, size=2000),
        np.random.uniform(-90, 89, size=2000)
    )
)

In [3]:
# Generating a list of cities using CitiPy from the random coordinate data
cities = []

for coord in coords:
    city = cp.nearest_city(coord[0], coord[1]).city_name
    
    if city not in cities:
        cities.append(city)

In [ ]:
# Creating a city data list to hold all city data
cities_data = []

# Incrementors and a 'purged' count for our progress display
i = 0
j = 0
purged = 0

# Text prefabs to scale in the progress display portion
buffer = " "
filler = "="

# Iterating through list of cities and outputting relevant data to a Pandas DataFrame
for city in cities:
    
    # Making API call for each city
    city_req = requests.get(
        "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key + "&q=" + city
    )
    
    # Confirming 200 code (All Clear!) before creating a data object to extract from
    if city_req.status_code == 200:
        city_info = city_req.json()
        
        #Extracting relevant data to our cities_data list
        cities_data.append(
            {
            "City": str(city_info["name"]),
            "Country": str(city_info["sys"]["country"]),
            "Latitude": float(city_info["coord"]["lat"]),
            "Longitude": float(city_info["coord"]["lon"]),
            "Max Temp": float(city_info["main"]["temp_max"]),
            "Humidity": int(city_info["main"]["humidity"]),
            "Cloudiness": int(city_info["clouds"]["all"]),
            "Wind Speed": float(city_info["wind"]["speed"]),
            "Current Description": str(city_info["weather"][0]["description"])
            }
        )
        
### LITERALLY EVERYTHING BELOW THIS POINT IS JUST ME HAVING FUN WITH THE DISPLAY ###

    else:
        # Removes bad city entry from cities list (slightly redundant) and
        # adds an extra purged list entry 
        purged += 1
        cities.remove(city)
               
    # Incrementor measuring total number of index iterated over is then tested as
    # a multiple of an arbitrary fraction of all values, which increments a var corresponding
    # to number of line pieces to print (divisor is # of "line pieces") -- allows for easy scaling 
    i += 1
    if (i % ((len(cities) - purged) / 40) < 1):
        j += 1
    
    # Prints a title, including the number of dropped cities and
    # Fills the progress bar as percentage ticks increase
    print(f"Extracted data from city #{len(cities_data)} of {len(cities) + purged} ({purged} Cities dropped).....   " 
          + "[" + (filler * j) + (buffer * (40 - j)) + "]", end="\r"
         )

print(f"\n\n\nAll cities logged. {len(cities_data)} extractions, {purged} drops.")   

In [5]:
# Creating a Pandas DataFrame from list of dictionaries
city_df = pd.DataFrame(cities_data)

#Applying formatting tweaks
city_df["Latitude"] = city_df["Latitude"].map("{:.2f}".format)
city_df["Longitude"] = city_df["Longitude"].map("{:.2f}".format)

#Displaying DataFrame to ensure all is up to spec
city_df.head(10)

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Gumdag,TM,39.21,54.59,82.62,11,36,14.92,scattered clouds
1,Jamestown,US,42.10,-79.24,60.44,42,100,4.61,overcast clouds
2,Saint-Philippe,RE,-21.36,55.77,71.19,66,55,7.25,broken clouds
3,Castro,BR,-24.79,-50.01,53.58,91,16,3.42,few clouds
4,Mar del Plata,AR,-38.00,-57.56,48.09,90,100,5.75,fog
5,Los Llanos de Aridane,ES,28.66,-17.92,65.95,79,40,14.97,scattered clouds
6,Hamilton,US,39.18,-84.53,75.56,62,0,9.22,clear sky
7,Ushuaia,AR,-54.80,-68.30,47.86,49,75,17.27,broken clouds
8,Jaraguá,BR,-15.76,-49.33,66.58,58,0,5.06,clear sky
9,Longyearbyen,SJ,78.22,15.64,44.44,87,40,3.44,scattered clouds


In [6]:
#Exporting the DataFrame to a .csv file in the Weather_Database folder
city_df.to_csv("Weather_Database/Weather_Database.csv")